In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_clamp_loss
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            nn.SiLU(),
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            nn.SiLU(),
        )
        self.input_proj = nn.Linear(config.dim_input, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
        )
        self.regr = nn.Sequential(
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_clamp_loss(y, labels, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_27_bigger_enc'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_27_bigger_enc-2024-11-27-09-53-06


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,

    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [6]:
trainer.train()
train_dataset.close()
val_dataset.close()

  0%|          | 0/150000 [00:00<?, ?it/s]

{'loss': 0.0072, 'grad_norm': 1.0319533348083496, 'learning_rate': 4.999666666666667e-05, 'epoch': 0.0}
{'loss': 0.0069, 'grad_norm': 0.0676129013299942, 'learning_rate': 4.9993333333333335e-05, 'epoch': 0.0}
{'loss': 0.0069, 'grad_norm': 0.27410101890563965, 'learning_rate': 4.999e-05, 'epoch': 0.0}
{'loss': 0.007, 'grad_norm': 0.08515193313360214, 'learning_rate': 4.9986666666666674e-05, 'epoch': 0.0}
{'loss': 0.0058, 'grad_norm': 0.05586791783571243, 'learning_rate': 4.998333333333334e-05, 'epoch': 0.0}
{'loss': 0.009, 'grad_norm': 0.751267671585083, 'learning_rate': 4.9980000000000006e-05, 'epoch': 0.0}
{'loss': 0.0075, 'grad_norm': 0.743998110294342, 'learning_rate': 4.997666666666667e-05, 'epoch': 0.0}
{'loss': 0.0073, 'grad_norm': 0.1352246254682541, 'learning_rate': 4.997333333333333e-05, 'epoch': 0.0}
{'loss': 0.0078, 'grad_norm': 0.8116639852523804, 'learning_rate': 4.997e-05, 'epoch': 0.0}
{'loss': 0.0065, 'grad_norm': 0.1373925656080246, 'learning_rate': 4.996666666666667e-

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.0031710134353488684, 'eval_runtime': 156.2624, 'eval_samples_per_second': 1599.873, 'eval_steps_per_second': 39.997, 'epoch': 1.0}
{'loss': 0.0038, 'grad_norm': 0.061585769057273865, 'learning_rate': 4.374666666666667e-05, 'epoch': 1.0}
{'loss': 0.0032, 'grad_norm': 0.7866349220275879, 'learning_rate': 4.374333333333334e-05, 'epoch': 1.0}
{'loss': 0.0028, 'grad_norm': 0.45405903458595276, 'learning_rate': 4.3740000000000005e-05, 'epoch': 1.0}
{'loss': 0.003, 'grad_norm': 0.8175208568572998, 'learning_rate': 4.373666666666667e-05, 'epoch': 1.0}
{'loss': 0.0021, 'grad_norm': 0.3931138217449188, 'learning_rate': 4.373333333333334e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.39174920320510864, 'learning_rate': 4.373e-05, 'epoch': 1.0}
{'loss': 0.0026, 'grad_norm': 0.21542789041996002, 'learning_rate': 4.372666666666667e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.09188255667686462, 'learning_rate': 4.3723333333333335e-05, 'epoch': 1.0}
{'loss': 0.004, 'grad_norm

  0%|          | 0/6250 [00:00<?, ?it/s]

...

{'eval_loss': 0.0018693896709010005, 'eval_runtime': 155.6314, 'eval_samples_per_second': 1606.36, 'eval_steps_per_second': 40.159, 'epoch': 6.0}
{'loss': 0.0023, 'grad_norm': 0.23783506453037262, 'learning_rate': 1.2496666666666668e-05, 'epoch': 6.0}
{'loss': 0.0021, 'grad_norm': 0.03289968892931938, 'learning_rate': 1.2493333333333333e-05, 'epoch': 6.0}
{'loss': 0.0018, 'grad_norm': 0.17657847702503204, 'learning_rate': 1.249e-05, 'epoch': 6.0}
{'loss': 0.0021, 'grad_norm': 0.2938689589500427, 'learning_rate': 1.2486666666666667e-05, 'epoch': 6.0}
{'loss': 0.002, 'grad_norm': 0.23100894689559937, 'learning_rate': 1.2483333333333335e-05, 'epoch': 6.0}
{'loss': 0.0018, 'grad_norm': 0.26092958450317383, 'learning_rate': 1.248e-05, 'epoch': 6.0}
{'loss': 0.0033, 'grad_norm': 0.37222784757614136, 'learning_rate': 1.2476666666666667e-05, 'epoch': 6.0}
{'loss': 0.0017, 'grad_norm': 0.043581534177064896, 'learning_rate': 1.2473333333333335e-05, 'epoch': 6.0}
{'loss': 0.0023, 'grad_norm': 0.0

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.0018594588618725538, 'eval_runtime': 154.9506, 'eval_samples_per_second': 1613.417, 'eval_steps_per_second': 40.335, 'epoch': 7.0}
{'loss': 0.0015, 'grad_norm': 0.1145099550485611, 'learning_rate': 6.2466666666666664e-06, 'epoch': 7.0}
{'loss': 0.0031, 'grad_norm': 0.23030322790145874, 'learning_rate': 6.243333333333333e-06, 'epoch': 7.0}
{'loss': 0.0015, 'grad_norm': 0.3239206075668335, 'learning_rate': 6.24e-06, 'epoch': 7.0}
{'loss': 0.0014, 'grad_norm': 0.11795128881931305, 'learning_rate': 6.236666666666667e-06, 'epoch': 7.0}
{'loss': 0.0018, 'grad_norm': 0.17458675801753998, 'learning_rate': 6.2333333333333335e-06, 'epoch': 7.0}
{'loss': 0.0015, 'grad_norm': 0.12012682855129242, 'learning_rate': 6.2300000000000005e-06, 'epoch': 7.0}
{'loss': 0.0012, 'grad_norm': 0.06981182098388672, 'learning_rate': 6.226666666666667e-06, 'epoch': 7.0}
{'loss': 0.0018, 'grad_norm': 0.060735028237104416, 'learning_rate': 6.223333333333334e-06, 'epoch': 7.0}
{'loss': 0.0013, 'grad_n

  0%|          | 0/6250 [00:00<?, ?it/s]

{'eval_loss': 0.001794710522517562, 'eval_runtime': 155.7331, 'eval_samples_per_second': 1605.311, 'eval_steps_per_second': 40.133, 'epoch': 8.0}
{'train_runtime': 7045.8946, 'train_samples_per_second': 851.56, 'train_steps_per_second': 21.289, 'train_loss': 0.0023851330907518663, 'epoch': 8.0}


In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [8]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [9]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]